In [344]:
"""
Обработка результатов опросника Сфера профессиональных предпочтений учащихся. (модификация Г. В. Резапкиной)
"""

'\nОбработка результатов опросника Сфера профессиональных предпочтений учащихся. (модификация Г. В. Резапкиной)\n'

In [345]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time
import re

In [346]:
file_data_xlsx = 'data/SPPU.xlsx'
path_to_end_folder = 'data'

In [347]:
def processing_result_sppu(row):
    """
    Обработка результатов тестирования
    """

    # Создаем словарь для хранения данных
    dct_type = {'сфера работы с людьми': 0, 'сфера умственного труда': 0, 'сфера технических интересов': 0,
                'сфера эстетики и искусства': 0,
                'сфера физического труда, подвижной деятельности': 0,
                'сфера материальных интересов, планово-экономических видов работ': 0}
    dct_error = {}  # словарь для хранения ошибочных  значений, для того чтобы было легче находить ошибки при обновлении

    # 1
    if row[0] == 'общаться с самыми разными людьми;':
        dct_type['сфера работы с людьми'] += 1
    elif row[0] == 'что-нибудь делать своими руками – мебель, одежду,машины и т.д.;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[0] == 'снимать фильмы, рисовать, писать книги, выступать на сцене и т.д.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error[1] = row[0]

    # 2
    if row[1] == 'художественная форма, мастерство писателя или режиссера;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[1] == 'сюжет, действие героев;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[1] == 'информация, которая может пригодиться в жизни.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error[2] = row[1]

    # 3
    if row[2] == 'в области науки;':
        dct_type['сфера умственного труда'] += 1
    elif row[2] == 'за общественную деятельность;':
        dct_type['сфера работы с людьми'] += 1
    elif row[2] == 'в области искусства.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error[3] = row[2]

    # 4
    if row[3] == 'управляющим банка;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[3] == 'главным инженером на производстве;':
        dct_type['сфера технических интересов'] += 1
    elif row[3] == 'начальником экспедиции.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error[4] = row[3]

    # 5
    if row[4] == 'достижения науки;':
        dct_type['сфера умственного труда'] += 1
    elif row[4] == 'развитие производства;':
        dct_type['сфера технических интересов'] += 1
    elif row[4] == 'взаимопонимание среди людей.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error[5] = row[4]

    # 6
    if row[5] == 'благоустройством школы (столовая, спортзал, компьютеры);':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[5] == 'созданием дружного, сплоченного коллектива;':
        dct_type['сфера работы с людьми'] += 1
    elif row[5] == 'разработкой новых технологий обучения.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error[6] = row[5]

    # 7
    if row[6] == 'внешний вид экспонатов (цвет, форма);':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[6] == 'внутреннее устройство экспонатов (механизм);':
        dct_type['сфера умственного труда'] += 1
    elif row[6] == 'практическое применение экспонатов.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error[7] = row[6]

    # 8
    if row[7] == 'мужество, смелость, выносливость;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[7] == 'дружелюбие, чуткость, отзывчивость;':
        dct_type['сфера работы с людьми'] += 1
    elif row[7] == 'ответственность, аккуратность.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error[8] = row[7]

    # 9
    if row[8] == 'писать стихи или музыку или рисовать;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[8] == 'ставить различные опыты;':
        dct_type['сфера умственного труда'] += 1
    elif row[8] == 'тренироваться.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error[9] = row[8]

    # 10
    if row[9] == 'экстремальный туризм (альпинизм, виндсерфинг, горные лыжи);':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[9] == 'деловое общение;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[9] == 'возможность знакомства с историей и культурой другой страны.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error[10] = row[9]

    # 11
    if row[10] == 'о машине нового типа;':
        dct_type['сфера технических интересов'] += 1
    elif row[10] == 'о новой научной теории;':
        dct_type['сфера умственного труда'] += 1
    elif row[10] == 'о человеческих взаимоотношениях.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error[11] = row[10]

    # 12
    if row[11] == 'технический;':
        dct_type['сфера технических интересов'] += 1
    elif row[11] == 'музыкальный;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[11] == 'спортивный.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error[12] = row[11]

    # 13
    if row[12] == 'улучшению взаимопонимания между учителями и учениками;':
        dct_type['сфера работы с людьми'] += 1
    elif row[12] == 'поддержанию здоровья учащихся, занятиям спортом;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[12] == 'укреплению дисциплины.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error[13] = row[12]

    # 14
    if row[13] == 'научно-популярные фильмы;':
        dct_type['сфера умственного труда'] += 1
    elif row[13] == 'программы о культуре и спорте;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[13] == 'спортивные программы.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error[14] = row[13]

    # 15
    if row[14] == 'с машинами, механизмами;':
        dct_type['сфера технических интересов'] += 1
    elif row[14] == 'с объектами природы;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[14] == 'с детьми или сверстниками.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error[15] = row[14]

    # 16
    if row[15] == 'давать знания и умения;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[15] == 'учить общению с другими людьми;':
        dct_type['сфера работы с людьми'] += 1
    elif row[15] == 'обучать навыкам работы.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error[16] = row[15]

    # 17
    if row[16] == 'вести здоровый образ жизни;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[16] == 'иметь возможность заниматься творчеством;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[16] == 'иметь удобные бытовые условия.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error[17] = row[16]

    # 18
    if row[17] == 'защита интересов и прав граждан;':
        dct_type['сфера работы с людьми'] += 1
    elif row[17] == 'забота о материальном благополучии людей;':
        dct_type['сфера технических интересов'] += 1
    elif row[17] == 'наука и технический прогресс.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error[18] = row[17]

    # 19
    if row[18] == 'физкультуры;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[18] == 'математики;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[18] == 'труда.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error[19] = row[18]

    # 20
    if row[19] == 'планировать производство продукции;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[19] == 'изготавливать изделия;':
        dct_type['сфера технических интересов'] += 1
    elif row[19] == 'заниматься сбытом продукции.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error[20] = row[19]

    # 21
    if row[20] == 'о выдающихся ученых и их открытиях;':
        dct_type['сфера умственного труда'] += 1
    elif row[20] == 'о творчестве ученых и музыкантов;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[20] == 'об интересных изобретениях.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error[21] = row[20]

    # 22
    if row[21] == 'делая что-то по хозяйству;':
        dct_type['сфера технических интересов'] += 1
    elif row[21] == 'с книгой;':
        dct_type['сфера умственного труда'] += 1
    elif row[21] == 'на выставках, концертах и пр.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error[22] = row[21]

    # 23
    if row[22] == 'о художественной выставке;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[22] == 'о ситуации на фондовой бирже;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[22] == 'о научном открытии.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error[23] = row[22]

    # 24
    if row[23] == 'в помещении, где много людей;':
        dct_type['сфера работы с людьми'] += 1
    elif row[23] == 'в необычных условиях;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[23] == 'в обычном кабинете.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error[24] = row[23]
    # проверяем на ошибки
    if len(dct_error) > 0:
        return dct_error
    else:
        # сортируем по убыванию
        result_lst = sorted(dct_type.items() , key=lambda t : t[1] , reverse=True)
        begin_str = 'Результат тестирования\n'
        # создаем строку с результатами
        for sphere,value in result_lst:
            begin_str += f'{sphere} - {value}\n'


        #добавляем описание
        return begin_str





In [348]:
df = pd.read_excel(file_data_xlsx)
threshold_sppu = 8 # количество колонок не относящихся к вопросам

In [349]:
base_df = df.iloc[:,:threshold_sppu] # создаем датафрейм с данными не относящимися к тесту

In [350]:
# делаем строковыми названия колонок
base_df.columns = list(map(str,base_df.columns))
# заменяем пробелы на нижнее подчеркивание и очищаем от пробельных символов в начале и конце
base_df.columns = [column.strip().replace(' ','_') for column in base_df.columns]

In [351]:
# очищаем от всех символов кроме букв цифр
base_df.columns = [re.sub(r'[^_\d\w]','',column) for column in base_df.columns]

In [352]:
answers_df = df.iloc[:,threshold_sppu:] # датафрейм с результатами

In [353]:
answers_df.columns = [f'Вопрос_ №_{i}' for i in range(1,answers_df.shape[1]+1)]

In [354]:
answers_df = answers_df.astype(str)

In [355]:
answers_df = answers_df.applymap(lambda x:x.strip())

In [356]:
answers_df['Результат'] = answers_df.apply(processing_result_sppu, axis=1)

answers_df['Результат'] = answers_df['Результат'].astype(str)

In [357]:
# соединяем после обработки
df = pd.concat([base_df,answers_df],axis=1)


In [358]:
# Создаем строку с описанием
output_str = """
Шкала оценки результатов
10-12 баллов – ярко выраженная профессиональная склонность;
7-9 баллов – средне выраженная профессиональная склонность;
4-6 баллов – слабо выраженная профессиональная склонность;
0-3 баллов – профессиональная склонность не выражена.

Примеры профессий для каждой сферы
Сфера работы с людьми: учитель, педагог, экскурсовод, воспитатель,  социолог, психолог, менеджер по персоналу, следователь.
Сфера умственного труда: ученый-исследователь (математик, физик, химик, кибернетик, археолог, геолог),  инженер, юрист, врач, эколог, архитектор, продюсер.
Сфера технических интересов:  программист, электротехник, радиотехник, Web-мастер, статистик, водитель, технолог, диспетчер, секретарь-машинистка, телефонист.
Сфера эстетики и искусства: художник, дизайнер, писатель, поэт,  режиссер, артист, конструктор, косметолог, костюмер, гример, кондитер, портной-кутюрье, цветовод. 
Сфера физического труда,  подвижной деятельности: спортсмен, фотограф, экспедитор, парикмахер,  бармен, официант, стюардесса, продавец, закройщик, специалист по ремонту, кассир, медперсонал, бригадир, кладовщик,  почтальон,  фермер,  водитель-дальнобойщик, полицейский, военный.
Сфера материальных интересов,  планово-экономических видов работ: экономист, администратор, менеджер, предприниматель, аудитор, специалист по рекламе, брокер, агент страховых компаний, коммерсант, завхоз.
"""

In [360]:
if 'ФИО' in df.columns:
    df['Описание_результата'] = df['ФИО'] + ' \nСфера профессиональных предпочтений учащихся\n' + df['Результат'] + output_str
else:
    df['Описание_результата'] ='Сфера профессиональных предпочтений учащихся\n' + df['Результат'] + output_str

In [361]:
# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)

df.to_excel(f'{path_to_end_folder}/Полная таблица с результатами СППУ от {current_time}.xlsx', index=False)

In [362]:

# Создаем сокращенный вариант
send_df = df.iloc[:,:threshold_sppu]
# Добавляем колонки с результатами
send_df['Результат'] = df['Результат']
send_df['Описание_результата'] = df['Описание_результата']

In [363]:



wb = openpyxl.Workbook()

for row in dataframe_to_rows(send_df, index=False, header=True):
    wb['Sheet'].append(row)

wb['Sheet'].column_dimensions['A'].width = 15
wb['Sheet'].column_dimensions['B'].width = 40
wb['Sheet'].column_dimensions['C'].width = 30
wb['Sheet'].column_dimensions['D'].width = 20
wb['Sheet'].column_dimensions['E'].width = 10
wb['Sheet'].column_dimensions['F'].width = 20
wb['Sheet'].column_dimensions['G'].width = 20
wb['Sheet'].column_dimensions['H'].width = 40

wb.save(f'{path_to_end_folder}/Таблица для рассылки с результатами СППУ  от {current_time}.xlsx')